AST Enumeration
=

This notebook adapts the AST form shown in 'Fast algorithms for classical specifications of stabiliser states and Clifford gates' https://arxiv.org/pdf/2311.10357.pdf

### The stabilizer state eqn

The equation used to find a stabilizer state from an AST is:

$$\left|\mathcal{K},q,l\right\rangle = \left|\Psi\right\rangle \propto \sum_{x \in \mathcal{K}}i^{l(x)}(-1)^{Q(x)}\left|x\right\rangle$$

Where:
* $\left|\Psi\right\rangle$ is a stabilizer state
* $\mathcal{L(K)}$ is a dimension $k$ linear subspace, $\mathcal{L(K)} \subseteq \mathbb{F}^{n}_{2}$, with basis $\{g^1,g^2,...,g^k\}$
* $\mathcal{K}$ is a dimension $k$ affine subspace, $\mathcal{K} \subseteq \mathbb{F}^{n}_{2}$, with $\mathcal{K} = \mathcal{L(K)} \oplus \vec{h}$
* $\vec{h}$ is a nonunique shift vector, $\vec{h} \in \mathcal{K}$
* $x \in \mathcal{K}$ is a point in the affine space, defined by a coordinate vector $\vec\alpha = (\alpha_1,\alpha_2,...,\alpha_k)$ as $x = \sum_{i=1}^k\alpha_ig^i$
* $l(x)$ is a linear form that maps the affine space to a binary value; $l(x): \mathcal{K}\mapsto \mathbb{F}_2$. It is defined by a binary valued $1\times k$ vector $\vec{b}$ as $l(x) = \vec{b}\cdot\vec{\alpha}$
* $Q(x)$ is a quadratic form that maps the affine space to a binary value; $Q(x): \mathcal{K}\mapsto \mathbb{F}_2$. It is defined by a binary valued $k\times k$ upper triangular matrix $J$ as $Q(x) = \vec{\alpha}J\vec{\alpha}^T$

---

### The Quadratic & Linear forms

The quadratic form, as stated above, uses an upper triangular $k\times k$ matrix $J$ with binary entries. This means for a given affine subspace with dimension $k$, there is a total of $2^{k(k+1)/2}$ different quadratic forms.

Similarly, the linear form is defined by a vector $\vec{b}$ of length $k$ with binary entries. This implies there are a total of $2^k$ different linear forms for a given affine subspace with dimension $k$.

---

### The Numbers Game

Using the information above, we can find out how many unique ways we may write the AST for different values of $n$:

$n=1$
* $k = 0; 2^{0(0+1)/2}\cdot 2^0 = 1$
* $k = 1; 2^{1(1+1)/2}\cdot 2^1 = 4$
* There being two affine subsets of dimension $k=0$, and one of $k=1$ gives total $= 6$

$n=2$
* $k = 0; 2^{0(0+1)/2}\cdot 2^0 = 1$
* $k = 1; 2^{1(1+1)/2}\cdot 2^1 = 4$
* $k = 2; 2^{2(2+1)/2}\cdot 2^2 = 32$
* There being four affine subsets of dimension $k=0$, and six of $k=1$ and one of $k=2$ gives total $= 60$

$n=3$
* $k = 0; 2^{0(0+1)/2}\cdot 2^0 = 1$
* $k = 1; 2^{1(1+1)/2}\cdot 2^1 = 4$
* $k = 2; 2^{2(2+1)/2}\cdot 2^2 = 32$
* $k = 3; 2^{3(3+1)/2}\cdot 2^3 = 512$
* Similar analysis to above gives total $= 1080$

The amount of unique ways to write the equation lines up perfectly with the number of stabilizer states we require for a given $n$.

---

In [ ]:
import numpy as np
import itertools as it
from sympy.utilities.iterables import multiset_permutations
from itertools import chain, combinations

In [ ]:
def Fp(p):
    Fp = []
    for element in range(p):
        Fp.append(element)
    return Fp

def Fnp(n,Fp):
    combi = list(list(it.product(Fp, repeat=n)))
    return combi

def powerset(sets):
    x = len(sets)
    res = []
    for i in range(1 << x):
        res.append(tuple([sets[j] for j in range(x) if (i & (1 << j))]))
    res.pop(0)
    return res

def closure2(subset):
    """This function checks if, given a set, addition mod2 is closed"""
    for i in range(0,len(subset)):
        for j in range(0,len(subset)):
            result = np.array(subset[i]) + np.array(subset[j])
            if tuple(result%2) not in subset: return False
    return True

def zero2(subset):
    """This function checks if the zero vector is contained in the subset"""
    zero = tuple([0]*len(subset[0]))
    if zero in subset: return True
    else: return False
    
def multi(subset):
    """This function checks if scalar multiplication mod2 is closed"""
    for i in range(0,len(subset)):
        zero = tuple([(z * 0)%2 for z in subset[i]])
        one = tuple([(z * 1)%2 for z in subset[i]])
        if zero not in subset:
            return False
        elif one not in subset:
            return False
        else:
            return True
        
def subspace2(subset):
    """This function checks if a space is a subspace"""
    return (closure2(subset) and zero2(subset) and multi(subset))

def subspaces2(subsets):
    """This function generates subspaces of a set"""
    spaces = []
    rejected = []
    for subset in subsets:
        if (subspace2(subset)==True):
            spaces.append(subset)
        else:
            rejected.append(subset)
    return spaces, rejected

def affine_subsets(n):
    F = Fnp(n,Fp(2))
    p = powerset(Fnp(n,Fp(2)))
    testsubspaces, testrejected = subspaces2(p)
    
    affine_subs = []
    for space in testsubspaces:
        for vec in F:
            affine_sub = []
            for vec2 in space:
                thing = tuple((np.array(vec) + np.array(vec2))%2)
                affine_sub.append(thing)
            affine_sub.sort()
            if tuple(affine_sub) not in affine_subs:
                affine_subs.append(tuple(affine_sub))
    for i in range(len(affine_subs)):
        affine_subs[i] = np.array(affine_subs[i])

                
    return affine_subs

In [ ]:
#this function evaluates the linear form l(z)
def l(b,α):
    if len(α)==0:
        return 0
    return np.dot(b,α)



#this function evaluates the quaratic form Q(z)
def Q(J,α):
    k = len(α)
    if k==0:
        return 0
    
    α = α[np.newaxis]
    αJ = np.dot(α,J)
    αJαT = np.dot(αJ,α.T)
    
    return αJαT



#this function creates a dictionary linking all possible state names and its correspinding index
#in state vector form for a given n
def state_dictmaker(n):
    state_names = []
    for tup in Fnp(n,["0", "1"]):
        name = ""
        for s in tup:
            name+=s
        state_names.append(name)
    
    state_dict = {}
    for i in range(len(state_names)):
        state_dict[state_names[i]] = i
    
    return state_dict

def Js_finder(k):             #this function finds all possible J matrices for the quadratic form for a given n
    if k == 0:                #an exception for k=0
        return [[[]]]
    
    cnum = int(0.5*k*(k+1))   #find the number of different binary combinations possible for J
    cs = [list(c) for c in Fnp(cnum,Fp(2))]  #generate the different binary combinations that will be inserted into J

    Js = []                   #initialise the list of Js
    for c in cs:
        J = [[0 for _ in range(k)] for _ in range(k)]   #for each binary combo c, a new J is initialised
        for i in range(len(J)):                         
            for j in range(len(J[i])):
                if i<=j:                                #only the upper diagonal entries of J are populated
                    J[i][j] = c.pop(0)
        Js.append(J)
    return Js                 #return the list of Js

In [ ]:
def AST_to_stab(K, b, J,state_dict):
    k = len(b)                      #find the dimension of the linear subspace
    n = len(K[0])                   #find the number of qubits of the stabilizer state
    
    vec_index = []                  #find the indices of the state vector to input values to
    for vec in K:
        thing = ""
        for s in vec:
            thing+=str(s)
        vec_index.append(state_dict[thing])
    
    αs = Fnp(k,Fp(2))               #generate the vector coordinates that will simply correspond in an orderly manner with -
    for i in range(len(αs)):        #- the x points in the subspace K
        αs[i] = np.array(αs[i])
    
    vec = [0]*2**n                  #initialise the state vector 
    tidy_num = []                   #start a list to append the phases of the basis vectors to
    if k==0:                        #another exception for k=0
        tidy_num.append(1)
        
    for α in αs:                             #loop over all α vectors, which correspond one to one with the x points in K
        num = (-1)**(Q(J,α))*(1j)**(l(b,α))  #calculate the number
        tidy_num.append(complex(num))        #append it to the list of phases
        
    for i in vec_index:             #set the appropriate values in the vector to those found in tidy_num
        vec[i] = tidy_num.pop(0)
        
    return vec                      #return the output

In [ ]:
def all_ASTs(n, check = True):
    count = 0
    state_dict = state_dictmaker(n)
    Ks = affine_subsets(n)
    
    for K in Ks:
        k = int(np.log2(len(K)))
        bs = [list(v) for v in Fnp(k,Fp(2))]
        Js = Js_finder(k)
        for J in Js:
            for b in bs:
                if check==True:
                    print("J =", J)
                    print("b =", b)
                    print(AST_to_stab(K, b, J,state_dict))
                    print()
                    print("______________")
                count+=1
    return count

In [ ]:
all_ASTs(1)

In [ ]:
all_ASTs(2)

In [ ]:
all_ASTs(3)

In [ ]:
all_ASTs(4,check=False)